In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *
DATA_DIR = '/work/users/k/4/k4thryn/Repos/OLD_EpSampling_Nov2024/data/'

pd.set_option('display.max_columns', None)

In [2]:
ts = '20241108-123515'

fpath = os.path.join(DATA_DIR,'processed',f'training_target_df_{ts}.csv')
df = pd.read_csv(fpath)
df = df[df.True_county_inc_deaths >= 0]

fpath = os.path.join(DATA_DIR,'processed',f'training_acs_df_{ts}.csv')
df_acs = pd.read_csv(fpath)
df_acs.set_index('Fips',inplace=True)
df = df.join(df_acs,on='Fips')

In [16]:
for x in df.columns:
    print(x)

Date
State_fips
Fips
Pop
State_pop
Ratio
Proj_state_inc_deaths
True_state_inc_deaths
True_county_cum_deaths
True_county_inc_deaths
Naive_proj_deaths
Naive_true_deaths
True_county_inc_deaths_tm_4
True_county_inc_deaths_tm_5
True_county_inc_deaths_tm_6
True_county_inc_deaths_tm_7
True_county_inc_deaths_tm_8
True_county_inc_deaths_tm_9
True_county_inc_deaths_tm_10
True_county_inc_deaths_tm_11
POP
POP_M
POP_F
POP_A0004
POP_A0509
POP_A1014
POP_A1517
POP_A1819
POP_A20
POP_A21
POP_A2224
POP_A2529
POP_A3034
POP_A3539
POP_A4044
POP_A4549
POP_A5054
POP_A5559
POP_A6061
POP_A6264
POP_A6566
POP_A6769
POP_A7074
POP_A7579
POP_A8084
POP_A85p
POP_A25p
EDU_ltHS
EDU_HS
EDU_ASSOC
EDU_BACH
EDU_MAST
EDU_PROF
EDU_DOCT
POP_x3
POP_NH
POP_HISP_x12
POP_A3564
HINS_A0018
HINS_A1934
HINS_A3564
HINS_A65p
HH_x5
HH_1P
HH_2P
HH_3P
HH_4P
HH_5P
HH_6P
HH_7pP
HH_x6
HHI_00_10k
HHI_10_15k
HHI_15_20k
HHI_20_25k
HHI_25_30k
HHI_30_35k
HHI_35_40k
HHI_40_45k
HHI_45_50k
HHI_50_60k
HHI_60_75k
HHI_75_100k
HHI_100_125k
HHI_125_150k
H

In [17]:
import statsmodels.api as sm
from epsampling.utils import get_chunks
from epsampling.modeling import get_date_chunked_splits, get_performance

def get_test_train_split(df):
    # Test-train split based on dates (weeks)
    # Train on 90 weeks, test on 9 weeks, chosen at random.
#     dts = list(df.Date.unique())
#     dts_test = random.sample(dts, 9)
#     dts_train = [x for x in dts if x not in dts_test]
#     df_train = df[df.Date.isin(dts_train)]
#     df_test = df[df.Date.isin(dts_test)] 
    # Based on 1-month chunks
    chunks = get_chunks(list(df.Date.unique()), num_membs=4)
    df_train, df_test = get_date_chunked_splits(df, chunks, 0)
    return df_train, df_test

In [26]:
def get_preds(df_train, df_test, feats):

    y_train = df_train['True_county_inc_deaths']
    y_test = df_test['True_county_inc_deaths']

    feats = ['True_state_inc_deaths']

    X_train = df_train[feats]
    
    X_test = df_test[feats]
    
    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)

    reg = sm.GLM(y_train, X_train).fit()
    
    preds = reg.predict(X_test)
    
    return preds

In [29]:
df_train, df_test = get_test_train_split(df)
# len(df_train), len(df_test)

feats = list(df_acs.columns) + ['Ratio','Proj_state_inc_deaths','Naive_proj_deaths','Pop']
preds = train_model(df_train, df_test, feats)
df_test['preds'] = preds

In [28]:
res_dfs = []
models = {}
chunks = get_chunks(list(df.Date.unique()), num_membs=8)

for ALG in ['GLM']:
    for i,test_chunk in tqdm(enumerate(chunks), total=len(chunks)):

        df_train, df_test = get_date_chunked_splits(df, chunks, i)
        
        df_test[f'Run'] = i
        
        PRED_COL = 'PRED county deaths'    
        df_res = get_df_res(df_train, df_test, X_COLS, Y_COL, PRED_COL, ALG)

        res_dfs.append(df_res)
    
df_full_res = pd.concat(res_dfs)
display(df_full_res)

In [12]:
df_train.Date.nunique()

94

In [13]:
df_test.Date.nunique()

5

In [ ]:

# random.seed(666)



dts = list(df.Date.unique())
dts_test = random.sample(dts, 9)
dts_train = [x for x in dts if x not in dts_test]
df_train = df[df.Date.isin(dts_train)]
df_test = df[df.Date.isin(dts_test)]



y_train = df_train['True_county_inc_deaths']
y_test = df_test['True_county_inc_deaths']

feats = ['True_state_inc_deaths']

X_train = df_train[feats]
X_train = sm.add_constant(X_train)
X_test = df_test[feats]
X_test = sm.add_constant(X_test)

reg = sm.GLM(y_train, X_train).fit()

print(reg.summary())

# df_test['pred'] = reg.predict(X_test)
# df_test['pred_abs_err'] = abs(df_test['pred'] - df_test['True_county_inc_deaths'])
# df_test['naive_abs_err'] = abs(df_test['Naive_proj_deaths'] - df_test['True_county_inc_deaths'])

df_train['pred'] = reg.predict(X_train)
df_train['pred_abs_err'] = abs(df_train['pred'] - df_train['True_county_inc_deaths'])
df_train['naive_abs_err'] = abs(df_train['Naive_proj_deaths'] - df_train['True_county_inc_deaths'])

# Summarize errors
# summary = {
#     'pois_err': (df_test['pred_abs_err']).sum(),
#     'naive_err': (df_test['naive_abs_err']).sum()
# }
# print(summary)

# summary = {
#     'pois_err': (df_test['pred_abs_err'] ** 2).sum(),
#     'naive_err': (df_test['naive_abs_err'] ** 2).sum()
# }
# print(summary)


# Summarize errors
summary = {
    'pois_err': (df_train['pred_abs_err']).sum(),
    'naive_err': (df_train['naive_abs_err']).sum()
}
print(summary)

summary = {
    'pois_err': (df_train['pred_abs_err'] ** 2).sum(),
    'naive_err': (df_train['naive_abs_err'] ** 2).sum()
}
print(summary)

In [ ]:
!pip install catboost


In [ ]:
model_names = [f'Deaths_{alg}', 'Deaths_naive']
model_preds = [df_test[alg], df_test['Naive']]
metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)

print(f'\n\n\n\n============ TEST MONTH {idx} ============\n')
for k,v in metrics_dict.items():
    print(f' > {k}')
    for k2,v2 in v.items():
        if k2=='r2':
            k3 = 'r-squared'
        else:
            k3 = k2
        print(f'  • {k3}: \t{round(v2,5)}')


In [ ]:
import statsmodels.api as sm
from statsmodels.discrete.count_model import ZeroInflatedPoisson

X = df[['Naive_proj_deaths']]
X = sm.add_constant(X)
y = df[['True_county_inc_deaths']]

# reg = ZeroInflatedPoisson(y, X, inflation="logit").fit()
reg = sm.GLM(y, X).fit()

print(reg.summary())

df['pois_pred'] = reg.predict(X)
df['pois_abs_err'] = abs(df['pois_pred'] - df['True_county_inc_deaths'])
df['naive_abs_err'] = abs(df['Naive_proj_deaths'] - df['True_county_inc_deaths'])

In [ ]:
# Summarize errors
summary = {
    'pois_err': (df['pois_abs_err']).sum(),
    'naive_err': (df['naive_abs_err']).sum()
}
print(summary)

summary = {
    'pois_err': (df['pois_abs_err'] ** 2).sum(),
    'naive_err': (df['naive_abs_err'] ** 2).sum()
}
print(summary)